In [28]:
#IMPORTS:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Embedding
from tensorflow.keras.optimizers import SGD

import os
import skimage.io as io
import skimage.transform as trans

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
#from keras import backend as keras

import rawpy
import cv2
import numpy as np
from numpy import genfromtxt

In [41]:
HEIGHT = 64
WIDTH = 64

In [42]:
def fixFile(file):
    return file.replace("data", "smalldata").replace("ARW", "csv")

In [43]:
def loadFile(file):
    good = fixFile(file)
    data = genfromtxt(good, delimiter=',')
    return data.reshape((HEIGHT, WIDTH,3))

In [44]:
valPath = "/home/takats_balint1/deephf/data/Sony_val_list.txt"
trainPath = "/home/takats_balint1/deephf/data/Sony_train_list.txt"
testPath = "/home/takats_balint1/deephf/data/Sony_test_list.txt"
replaceString = "/home/takats_balint1/deephf/data"

In [58]:
saveSmall(loadFileNames(testPath))

In [59]:
saveSmall(loadFileNames(valPath))

In [60]:
saveSmall(loadFileNames(trainPath))

In [55]:
def loadFileNames(path):
    returnArray = []
    with open(path) as File:
        for line in File:
            data, truth, _, _ = line.split()
            data = data.replace(".", replaceString, 1)
            truth = truth.replace(".", replaceString, 1)
            # This method is slow but we only have a 1000 lines so this quick and dirty algortihm will do just fine
            returnArray.append([data, truth])
    
    return returnArray

In [56]:
def saveFiles(listOfFiles):
    for i in listOfFiles:
        raw = rawpy.imread(i)
        raw = raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
        raw = np.float32(raw / 65535.0)
        raw = cv2.resize(raw, dsize=(HEIGHT, WIDTH), interpolation=cv2.INTER_CUBIC)
        raw = np.asarray(raw)
        name = i
        raw.tofile(fixFile(name),sep=',',format='%10.8f')

In [57]:
def saveSmall(fileNames):
    saveFiles([i[0] for i in fileNames])
    saveFiles(list(set([i[1] for i in fileNames])))
    